In [23]:
from __future__ import print_function

import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import savgol_filter


from six.moves import xrange

import umap

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from sklearn import preprocessing
from torchsummary import summary

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:
spectogram_array_path_trim_30db =  '/home/ubuntu/speech2speech/data/interim/spectogram_array_trim_30db'

In [26]:
import os
import glob

files_np = list(glob.glob(os.path.join(spectogram_array_path_trim_30db,'*.*')))

In [63]:
np.array([1,2,3,4,5,6,7,8,9])[::3]

array([1, 4, 7])

In [57]:
xt = torch.randn(1025,100)

In [58]:
xt.shape

torch.Size([1025, 100])

In [59]:
xt.unsqueeze(0).shape

torch.Size([1, 1025, 100])

In [61]:
xt.unsqueeze(0).repeat(3,1,1).shape

torch.Size([3, 1025, 100])

In [48]:
def spectrograms_to_torch_dataset(files_np, max_col):
    """
    Generates torch tensors for the spectrogram array and return
    them as a dataset
    """
    X_list = []
    Y_list = []
    for file in files_np:
        load_x = np.load(file)
        x = np.zeros((load_x.shape[0], max_col), dtype=np.float32)
        c = min(max_col, load_x.shape[1])
        x[:load_x.shape[0], :c] = load_x[:, :c]

        x = torch.tensor(x, device='cpu').float()
        x = x.unsqueeze(0)
        trial_y = file.split('trim_spec_p')[1][:3]
        trial_y = torch.tensor(int(trial_y), device='cpu').float()
        X_list.append(x)
        Y_list.append(trial_y)

    x_Tensor = torch.stack(X_list)
    y_Tensor = torch.stack(Y_list)
    tensordataset = torch.utils.data.TensorDataset(x_Tensor, y_Tensor)
    return tensordataset

In [49]:
full_tensordataset =  spectrograms_to_torch_dataset(files_np[:10], 100)

In [50]:
def split_dataset(full_tensordataset, train_data_fraction=0.8,
                  validation_data_fraction=0.1):
    train_val_data_fraction = train_data_fraction + validation_data_fraction
    assert(train_val_data_fraction < 1.0, 'Check data split fractions')
    train_val_size = int(train_val_data_fraction * len(full_tensordataset))
    test_size = len(full_tensordataset) - train_val_size
    train__val_dataset, test_dataset = torch.utils.data.random_split(
        full_tensordataset, [train_val_size, test_size])
    train_size = int(
        (train_data_fraction /
         train_val_data_fraction) *
        len(train__val_dataset))
    val_size = len(train__val_dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        train__val_dataset, [train_size, val_size])

    assert(len(test_dataset) + len(train_dataset) + len(val_dataset)
           == len(full_tensordataset), 'Data not split correctly ')
    return train_dataset, val_dataset, test_dataset

<ipython-input-50-9f4622dd8242>:4: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(train_val_data_fraction < 1.0, 'Check data split fractions')
<ipython-input-50-9f4622dd8242>:17: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(test_dataset) + len(train_dataset) + len(val_dataset)


In [51]:
def train_val_data_loaders(train_dataset, val_dataset, batch_size):
    training_loader = DataLoader(train_dataset,
                                 batch_size=batch_size,
                                 shuffle=True,
                                 pin_memory=True)
    validation_loader = DataLoader(val_dataset,
                                   batch_size=batch_size,
                                   shuffle=True,
                                   pin_memory=True)
    return training_loader, validation_loader

In [52]:
train_dataset, val_dataset, test_dataset = split_dataset(full_tensordataset, train_data_fraction=0.8,
                  validation_data_fraction=0.1)

In [56]:
train_dataset[0][0].shape

torch.Size([1, 1025, 100])

In [3]:
class PixelTransform:
    def __init__(self):
        pass

    def __call__(self, input):
        ar = np.array(input)

        return torch.from_numpy(ar).long()

In [4]:
dataset = datasets.MNIST('.', transform=PixelTransform(), download=True)

  0%|          | 16384/9912422 [00:00<01:08, 145345.76it/s]

9920512it [00:00, 40189429.12it/s]                           


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


32768it [00:00, 916620.23it/s]
1654784it [00:00, 12739236.29it/s]                           

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw
Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



8192it [00:00, 418985.43it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


In [13]:
dataset[0][0].shape

torch.Size([28, 28])

In [16]:
dataset[0]

(tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,
           18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,

In [18]:
F.one_hot(dataset[0][0], 256).shape

torch.Size([28, 28, 256])

In [21]:
F.one_hot(dataset[0][0], 256).permute(0, 3, 1, 2)

RuntimeError: number of dims don't match in permute